# Chapter 6: Advanced and Interactive Data Visualization Techniques

## Introduction

In Chapter 5, we learned how to create static visualizations using Matplotlib and Seaborn. While static charts are excellent for reports and publications, they have limitations when it comes to **exploration** and **discovery**.

This chapter introduces **interactive visualization**—charts that respond to user actions like hovering, clicking, zooming, and filtering. Interactive visuals help you (and your audience) explore data more deeply and answer questions on the fly.

### What You'll Learn
By the end of this chapter, you will be able to:
- Understand when to use static vs. interactive visualizations
- Create interactive charts with **Plotly**, **Bokeh**, and **Altair**
- Add controls like dropdowns and sliders to your charts
- Build simple dashboard-style layouts
- Create geographic (map) visualizations
- Export interactive charts for sharing

### Libraries Covered
| Library | Strengths |
|---------|-----------|
| **Plotly** | Easy interactive charts, great for sharing as HTML |
| **Bokeh** | Python-first, powerful for dashboards and apps |
| **Altair** | Clean declarative syntax, excellent for exploratory analysis |

> **Note**: You don't need to master all three libraries. Learn the concepts first, then pick the one that best fits your workflow.

## Prerequisites
You should be comfortable with:
- Basic Python and functions
- Pandas DataFrames
- Basic plotting concepts (from Chapter 5)

If a library isn't installed, you'll see a friendly message and a suggested install command.

In [ ]:
# Chapter 6 setup: imports and environment checks
from __future__ import annotations

import sys
import textwrap

def explain_install(package_name: str, extra: str | None = None) -> None:
    msg = f"Missing package: {package_name}."
    cmd = f"{sys.executable} -m pip install {package_name}"
    if extra:
        cmd = f"{sys.executable} -m pip install {package_name} {extra}"
    print(msg)
    print('Install with:')
    print(cmd)

print('Python:', sys.version.split()[0])

## Dataset used in this chapter
To keep this chapter self-contained, we'll use a built-in dataset from Plotly (Gapminder).

Why this dataset?
- It's small enough to explore quickly
- It has time, numeric measures, and categories
- It also includes country names for map examples

In [ ]:
import numpy as np
import pandas as pd

try:
    import plotly.express as px
except ImportError:
    explain_install('plotly')
    raise

df = px.data.gapminder()
df.head()

In [ ]:
df.info()

---
## 6.1 Limitations of Static Visualizations

Static charts (like a PNG image) are great for printed reports and quick summaries. But they can be limiting when you want to:
- **Explore details** — "What is the exact value at this point?"
- **Zoom into dense areas** — see patterns hidden in crowded regions
- **Filter categories interactively** — focus on one group at a time
- **Compare many groups** — without making dozens of separate charts

### When to Use Static vs. Interactive

| Use Case | Best Choice |
|----------|-------------|
| Printed reports, PDFs | Static |
| Presentations (PowerPoint) | Static |
| Data exploration | Interactive |
| Web dashboards | Interactive |
| Sharing with non-technical audience | Depends on context |

> **Common Beginner Mistake**: Trying to put *everything* into one static plot, which becomes cluttered and hard to read. Interactive charts can help here by letting users focus on what they care about.

Let's build a simple static chart first, then compare it to an interactive version.

In [ ]:
# Static example (Matplotlib): you can see the trend, but exploration is limited
import matplotlib.pyplot as plt

country = 'Canada'
subset = df[df['country'] == country].sort_values('year')

plt.figure(figsize=(7, 4))
plt.plot(subset['year'], subset['lifeExp'], marker='o')
plt.title(f'Life Expectancy Over Time (Static) — {country}')
plt.xlabel('Year')
plt.ylabel('Life Expectancy')
plt.grid(True, alpha=0.3)
plt.show()

### Interactive version (same data)
With an interactive chart, you can usually:
- Hover to see exact values
- Zoom and pan
- Export from the chart toolbar

Tip: In VS Code notebooks, Plotly figures typically render automatically. If you ever see a blank output, try rerunning the cell or updating Plotly.

In [ ]:
fig = px.line(subset, x='year', y='lifeExp', markers=True, title=f'Life Expectancy Over Time (Interactive) — {country}')
fig.update_layout(xaxis_title='Year', yaxis_title='Life Expectancy')
fig.show()

---
## 6.2 Interactive plotting concepts
Interactive visualizations usually combine **data + graphics + user actions**.

### Common interactive actions
- **Hover tooltips**: show exact values
- **Zoom / pan**: focus on a region
- **Selection / brushing**: highlight points by dragging a box
- **Filtering controls**: dropdowns, sliders, buttons
- **Linked views**: selecting data in one chart updates another

### Two important mental models
1. **Client-side interactivity**: the browser can handle it without a server (fast, simple).
2. **Server-backed interactivity**: user actions trigger Python code on a server (powerful, used in dashboards/apps).

Beginner warning: interactive charts are not automatically “better.” They can confuse readers if there’s no clear purpose. Always ask: *What question will this interactivity help someone answer?*

---
## 6.3 Interactive charts with Plotly (quick wins)
Plotly Express (`plotly.express` / `px`) lets you create interactive charts with very little code.

We'll build a scatter plot where:
- Each point is a country
- Size represents population
- Color represents continent
- Hover shows details

This is a classic interactive exploration chart.

In [ ]:
year = 2007
df_2007 = df[df['year'] == year].copy()

fig = px.scatter(
    df_2007,
    x='gdpPercap',
    y='lifeExp',
    size='pop',
    color='continent',
    hover_name='country',
    log_x=True,
    title=f'Gapminder {year}: Wealth vs Life Expectancy (Interactive)',
    labels={'gdpPercap': 'GDP per Capita (log scale)', 'lifeExp': 'Life Expectancy'}
)
fig.show()

### Customizing interactivity (tooltips)
Plotly lets you control what appears on hover. A common mistake is showing *too much* information.

We'll show only a few useful fields and format large numbers.

In [ ]:
fig = px.scatter(
    df_2007,
    x='gdpPercap',
    y='lifeExp',
    size='pop',
    color='continent',
    hover_name='country',
    hover_data={
        'continent': True,
        'pop': ':,',
        'gdpPercap': ':.0f',
        'lifeExp': ':.1f'
    },
    log_x=True,
    title=f'Better Hover Tooltips — {year}',
)
fig.show()

### Animation: exploring change over time
If your data has a time column, animation can help you explore trends.

Beginner warning: animations look impressive, but they can hide details if they move too fast. Use them mainly for exploration, and consider static summaries for final reporting.

In [ ]:
fig = px.scatter(
    df,
    x='gdpPercap',
    y='lifeExp',
    animation_frame='year',
    animation_group='country',
    size='pop',
    color='continent',
    hover_name='country',
    log_x=True,
    range_x=[100, 100000],
    range_y=[20, 90],
    title='Animated: Wealth vs Life Expectancy Over Time'
)
fig.show()

---
## 6.4 Interactive charts and controls (dropdowns and sliders)
Controls help users answer *their* questions without you writing a new chart each time.

We'll build one chart and add a dropdown that changes the **y-axis** metric.

This is a common dashboard technique: one control, many views.

In [ ]:
import plotly.graph_objects as go

metrics = {
    'Life Expectancy': 'lifeExp',
    'GDP per Capita': 'gdpPercap',
    'Population': 'pop'
}

x = df_2007['country']
y_default = df_2007[metrics['Life Expectancy']]

fig = go.Figure()
fig.add_trace(go.Bar(x=x, y=y_default, name='Value'))

buttons = []
for label, col in metrics.items():
    buttons.append({
        'label': label,
        'method': 'update',
        'args': [
            {'y': [df_2007[col]]},
            {'yaxis': {'title': label}, 'title': f'{label} by Country ({year})'}
        ]
    })

fig.update_layout(
    title=f'Life Expectancy by Country ({year})',
    xaxis_title='Country',
    yaxis_title='Life Expectancy',
    updatemenus=[{'type': 'dropdown', 'buttons': buttons, 'x': 1.02, 'y': 1.0}],
    height=500,
    margin=dict(l=40, r=200, t=60, b=120)
)

# Tip: bar charts with many categories can be crowded—zooming helps, but consider sorting or filtering in dashboards.
fig.update_xaxes(tickangle=60)
fig.show()

### Exercise 1 (quick practice): make it easier to read
Improve the chart above:
1. Sort countries by the selected metric (hint: you’ll need to rebuild `x` and `y` for each button).
2. Filter to a single continent using a variable like `continent = 'Asia'`.

Try it below. A starter cell is provided.

In [ ]:
# Exercise 1 starter
continent = 'Asia'
df_ex = df_2007[df_2007['continent'] == continent].copy()

# TODO: pick a metric and sort
metric_label = 'GDP per Capita'
metric_col = metrics[metric_label]

df_ex = df_ex.sort_values(metric_col, ascending=False)

fig = px.bar(
    df_ex,
    x='country',
    y=metric_col,
    title=f'{metric_label} in {continent} ({year})',
    labels={metric_col: metric_label, 'country': 'Country'}
)
fig.update_xaxes(tickangle=60)
fig.show()

---
## 6.5 Dashboard Fundamentals

A **dashboard** is a collection of visualizations (and usually controls) that supports a set of decisions. Think of it as a "control panel" for data.

### Key Components of a Dashboard

| Component | Purpose | Example |
|-----------|---------|---------|
| **KPIs / Metrics** | Quick summary numbers | "Total Revenue: $1.2M" |
| **Charts** | Visual patterns and comparisons | Bar, line, scatter plots |
| **Filters** | Let users focus on subsets | Dropdown for region, date slider |
| **Layout** | Organize information logically | Grid of cards, sections |

### Dashboard Design Principles

1. **Start with questions** — What decisions will this dashboard support?
2. **Know your audience** — Executives need summaries; analysts need details
3. **Less is more** — 2–6 visuals that answer specific questions
4. **Consistent design** — Use the same colors, fonts, and scales throughout
5. **Performance matters** — Slow dashboards don't get used

### Common Dashboard Mistakes

> **Beginner Warning**: Dashboards fail most often because they try to answer *too many* questions at once. Start with one decision-maker and 2–3 key questions.

- ❌ Cramming 20 charts onto one screen
- ❌ No clear story or purpose
- ❌ Inconsistent colors across charts
- ❌ Filters that don't apply to all relevant charts
- ✅ Focus on actionable insights

Below, we'll build a small "dashboard-like" view: multiple charts on one screen that share the same filtered dataset.

In [ ]:
from plotly.subplots import make_subplots

continent = 'Europe'
df_dash = df_2007[df_2007['continent'] == continent].copy()

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        f'Life Expectancy vs GDP per Capita ({continent}, {year})',
        f'Top 10 Countries by Population ({continent}, {year})'
    )
)

# Left: scatter
fig.add_trace(
    go.Scatter(
        x=df_dash['gdpPercap'],
        y=df_dash['lifeExp'],
        mode='markers',
        text=df_dash['country'],
        hovertemplate='Country=%{text}<br>GDP=%{x:.0f}<br>LifeExp=%{y:.1f}<extra></extra>',
        marker=dict(size=np.clip(df_dash['pop'] / 1_000_000, 6, 40), opacity=0.7)
    ),
    row=1, col=1
)

# Right: bar
top10 = df_dash.nlargest(10, 'pop')
fig.add_trace(
    go.Bar(x=top10['country'], y=top10['pop'], hovertemplate='%{x}<br>Pop=%{y:,}<extra></extra>'),
    row=1, col=2
)

fig.update_layout(height=450, showlegend=False, title_text=f'Dashboard View (Notebook) — {continent}, {year}')
fig.update_xaxes(type='log', title_text='GDP per Capita (log)', row=1, col=1)
fig.update_yaxes(title_text='Life Expectancy', row=1, col=1)
fig.update_xaxes(tickangle=45, row=1, col=2)
fig.update_yaxes(title_text='Population', row=1, col=2)
fig.show()

### From notebooks to real dashboards
In practice, dashboards are often built with app frameworks. Common choices:
- **Plotly Dash** (Python web apps around Plotly charts)
- **Streamlit** (simple Python apps, great for beginners)
- **Panel** (Bokeh ecosystem, flexible)

We won't launch a server in this chapter, but later you can turn your notebook visuals into an app.

Optional resource links:
- Dash: https://dash.plotly.com/
- Streamlit: https://streamlit.io/
- Panel: https://panel.holoviz.org/

---
## 6.6 Geospatial Visualization Basics

Geospatial charts display data on maps. They're powerful for showing regional patterns but come with unique challenges.

### Types of Map Visualizations

| Type | Description | Best For |
|------|-------------|----------|
| **Scatter Map** | Points placed at locations | City-level data, events |
| **Choropleth** | Regions colored by value | Country/state comparisons |
| **Bubble Map** | Points with size encoding | Population, magnitude data |
| **Heatmap** | Density coloring | Concentration patterns |

### Common Beginner Pitfalls
1. **Mixing up latitude and longitude** — Latitude is vertical (N/S), longitude is horizontal (E/W)
2. **Wrong geographic level** — Make sure your data matches (country vs. city vs. region)
3. **Misleading color scales** — Choose scales that don't distort perception
4. **Ignoring projections** — All flat maps distort reality; Mercator makes Greenland look huge!
5. **Using raw totals** — Large countries dominate; prefer per-capita or density measures

> **Tip**: Plotly has built-in support for country codes (ISO Alpha-3), so you don't always need latitude/longitude coordinates.

We'll use Plotly's built-in mapping support without any API keys.

In [ ]:
# Map 1: scatter_geo (points on a world map)
# Gapminder doesn't have lat/lon columns, but Plotly can use country codes (iso_alpha) with 'locations'
# For scatter_geo without lat/lon, we use 'locations' and 'locationmode'

fig = px.scatter_geo(
    df_2007,
    locations='iso_alpha',
    size='pop',
    color='continent',
    hover_name='country',
    projection='natural earth',
    title=f'Countries as Points (Population Size) — {year}',
    size_max=50
)
fig.update_layout(height=500)
fig.show()

### Choropleth maps (color by value)
A choropleth colors geographic regions (like countries) based on a value.

Tip: choropleths work best when you're comparing **rates** (e.g., per capita) rather than raw totals, because larger regions can dominate visually.

We'll color by GDP per capita.

In [ ]:
fig = px.choropleth(
    df_2007,
    locations='iso_alpha',
    color='gdpPercap',
    hover_name='country',
    color_continuous_scale='Viridis',
    title=f'Choropleth: GDP per Capita — {year}',
    labels={'gdpPercap': 'GDP per Capita'}
)
fig.update_layout(height=500)
fig.show()

### Exercise 2: ask a map question
Pick one of these tasks:
- Make a choropleth colored by **life expectancy**
- Make a choropleth colored by **population** (then discuss why that can be misleading)

Starter cell below.

In [ ]:
# Exercise 2 starter
value_col = 'lifeExp'  # try 'pop' or 'lifeExp'
title = {'lifeExp': 'Life Expectancy', 'pop': 'Population', 'gdpPercap': 'GDP per Capita'}.get(value_col, value_col)

fig = px.choropleth(
    df_2007,
    locations='iso_alpha',
    color=value_col,
    hover_name='country',
    color_continuous_scale='Plasma',
    title=f'Choropleth: {title} — {year}',
    labels={value_col: title}
)
fig.update_layout(height=500)
fig.show()

---
## 6.7 Beyond Plotly: Bokeh and Altair (quick introductions)
Different libraries are good for different workflows:
- **Plotly**: great general-purpose interactivity, easy sharing as HTML
- **Bokeh**: powerful Python-first interactivity, good for app-style dashboards
- **Altair**: clean “grammar of graphics” approach, great for exploratory analysis and selections

You do **not** need to master all of them. Learn the concepts, then pick one library that fits your use case.

### Bokeh example: interactive hover
Bokeh charts can include hover tools and are often used in dashboard-style apps.

If Bokeh isn't installed, install it with: `python -m pip install bokeh`

In [ ]:
try:
    from bokeh.io import output_notebook, show
    from bokeh.models import ColumnDataSource, HoverTool
    from bokeh.plotting import figure
except ImportError:
    explain_install('bokeh')
    raise

output_notebook()

data = df_2007[['country', 'gdpPercap', 'lifeExp', 'pop', 'continent']].copy()
data['pop_millions'] = data['pop'] / 1_000_000
source = ColumnDataSource(data)

p = figure(
    title=f'Bokeh: GDP vs Life Expectancy ({year})',
    x_axis_type='log',
    width=800,
    height=420,
    tools='pan,wheel_zoom,box_zoom,reset,save'
)

p.circle('gdpPercap', 'lifeExp', size='pop_millions', source=source, alpha=0.5)

hover = HoverTool(tooltips=[
    ('Country', '@country'),
    ('Continent', '@continent'),
    ('GDP per Capita', '@gdpPercap{0,0}'),
    ('Life Expectancy', '@lifeExp{0.0}'),
    ('Population', '@pop{0,0}')
])
p.add_tools(hover)

p.xaxis.axis_label = 'GDP per Capita (log scale)'
p.yaxis.axis_label = 'Life Expectancy'

show(p)

### Altair example: selection and brushing
Altair is built around a “describe what you want” style. It’s great for clean exploratory charts.

If Altair isn't installed, install it with: `python -m pip install altair`

Note: Altair outputs Vega-Lite specs. In some environments you may need an extra renderer; VS Code usually works out of the box.

In [ ]:
try:
    import altair as alt
except ImportError:
    explain_install('altair')
    raise

alt.data_transformers.disable_max_rows()

brush = alt.selection_interval()

base = alt.Chart(df_2007).properties(width=700, height=350)

points = (
    base.mark_circle(size=80)
    .encode(
        x=alt.X('gdpPercap:Q', scale=alt.Scale(type='log'), title='GDP per Capita (log)'),
        y=alt.Y('lifeExp:Q', title='Life Expectancy'),
        color=alt.condition(brush, 'continent:N', alt.value('lightgray')),
        tooltip=['country:N', 'continent:N', alt.Tooltip('gdpPercap:Q', format=',.0f'), alt.Tooltip('lifeExp:Q', format='.1f')]
    )
    .add_params(brush)
)

bars = (
    base.mark_bar()
    .encode(
        y=alt.Y('continent:N', title=None),
        x=alt.X('count():Q', title='Selected Countries'),
        color='continent:N'
    )
    .transform_filter(brush)
)

chart = alt.vconcat(points, bars).resolve_scale(color='independent').properties(title=f'Altair: Brush to Filter ({year})')
chart

---
## 6.8 Exporting interactive visualizations
Exporting matters because not everyone reads notebooks. Common export targets:
- **HTML** (best for interactive charts)
- **PNG/SVG/PDF** (static images for slides and PDFs)

Beginner warning: interactive charts do not embed nicely into PDFs by default. Usually you export a static image for PDFs.

In [ ]:
# Export a Plotly figure to HTML
export_fig = px.scatter(df_2007, x='gdpPercap', y='lifeExp', color='continent', hover_name='country', log_x=True,
                        title=f'Export Demo — {year}')

html_path = 'chapter06_export_demo_plotly.html'
export_fig.write_html(html_path, include_plotlyjs='cdn')
html_path

### Export to an image (optional)
Plotly can export static images using the `kaleido` engine. If you need PNG/SVG export, install kaleido:

`python -m pip install kaleido`

We'll try to export; if `kaleido` isn't installed, you'll get a helpful message.

In [ ]:
png_path = 'chapter06_export_demo_plotly.png'
try:
    export_fig.write_image(png_path, width=900, height=500, scale=2)
    print('Wrote:', png_path)
except Exception as e:
    print('Could not export image. This usually means kaleido is missing.')
    print(f'Install with: {sys.executable} -m pip install kaleido')
    print('Error:', e)

---
## 6.9 Embedding visualizations in reports and apps
### Embedding in reports
- **Notebook / HTML report**: easiest—keep interactivity
- **PDF / Word / PowerPoint**: typically export a static image

### Embedding in apps
Most Python dashboard frameworks embed charts in a web UI. Common patterns:
- Use a filter (dropdown/slider) to update charts
- Show a few key metrics (KPIs)
- Provide drill-down (clicking a chart reveals details)

We'll demonstrate a simple embedding workflow: generate an HTML snippet you could insert into a web page.

In [ ]:
# Convert a Plotly figure to an HTML snippet (useful for embedding)
html_snippet = export_fig.to_html(full_html=False, include_plotlyjs='cdn')
print('HTML snippet length (characters):', len(html_snippet))
print(html_snippet[:300] + '\n...')

### Exercise 3: Create an Exportable Chart

Practice exporting by:
1. Create an interactive scatter plot comparing any two numeric variables from the Gapminder data
2. Export it as an HTML file with a descriptive filename
3. (Optional) If you have kaleido installed, also export as PNG

Starter cell below.

In [ ]:
# Exercise 3 starter
# TODO: Create your own interactive chart and export it

# Example: Population vs GDP scatter
my_fig = px.scatter(
    df_2007,
    x='pop',
    y='gdpPercap',
    color='continent',
    hover_name='country',
    log_x=True,
    log_y=True,
    title='Exercise 3: Population vs GDP per Capita',
    labels={'pop': 'Population (log)', 'gdpPercap': 'GDP per Capita (log)'}
)

# Export to HTML
my_fig.write_html('exercise3_my_chart.html', include_plotlyjs='cdn')
print('Exported: exercise3_my_chart.html')

my_fig.show()

---
## Mini-Project: Build an Interactive Data Explorer

**Goal**: Create a simple interactive explorer for the Gapminder dataset where you can:
- Choose a **continent** to focus on
- Choose a **metric** (life expectancy, GDP per capita, or population)
- See a bar chart for a **selected year**

### Why This Matters
This exercise teaches a key real-world skill: building a **repeatable analysis tool** that others (or your future self) can explore without modifying code.

### Implementation Approaches
We'll show two versions:
1. **Interactive widgets** (using `ipywidgets`) — dropdowns and sliders that update the chart live
2. **Manual fallback** — change variables and rerun the cell (works everywhere)

Let's start by creating the core charting function:

In [ ]:
def make_explorer_chart(data: pd.DataFrame, metric_col: str, metric_label: str, year_value: int, continent_value: str) -> None:
    subset = data[(data['year'] == year_value) & (data['continent'] == continent_value)].copy()
    subset = subset.sort_values(metric_col, ascending=False)

    fig = px.bar(
        subset,
        x='country',
        y=metric_col,
        title=f'{metric_label} — {continent_value}, {year_value}',
        labels={'country': 'Country', metric_col: metric_label},
    )
    fig.update_xaxes(tickangle=60)
    fig.update_layout(height=500)
    fig.show()

available_years = sorted(df['year'].unique())
continents = sorted(df['continent'].unique())
metric_options = {
    'Life Expectancy': 'lifeExp',
    'GDP per Capita': 'gdpPercap',
    'Population': 'pop'
}

In [ ]:
# Version 1: ipywidgets (interactive controls)
try:
    import ipywidgets as widgets
    from IPython.display import display
except ImportError:
    explain_install('ipywidgets')
    widgets = None

if widgets is not None:
    year_widget = widgets.SelectionSlider(options=available_years, value=2007, description='Year', continuous_update=False)
    continent_widget = widgets.Dropdown(options=continents, value='Asia', description='Continent')
    metric_widget = widgets.Dropdown(options=list(metric_options.keys()), value='GDP per Capita', description='Metric')

    ui = widgets.HBox([continent_widget, metric_widget, year_widget])
    display(ui)

    def _update(continent: str, metric_label: str, year_value: int):
        make_explorer_chart(df, metric_options[metric_label], metric_label, year_value, continent)

    out = widgets.interactive_output(
        _update,
        {
            'continent': continent_widget,
            'metric_label': metric_widget,
            'year_value': year_widget
        }
    )
    display(out)

### Mini-project fallback (no widgets)
If `ipywidgets` is not available, use this simple workflow:
1. Change the variables
2. Rerun the cell

This is less interactive, but the *analysis structure* is the same.

In [ ]:
# Manual fallback controls
continent_value = 'Africa'
year_value = 2007
metric_label = 'Life Expectancy'

make_explorer_chart(df, metric_options[metric_label], metric_label, year_value, continent_value)

### Mini-project extension ideas (optional)
Try one improvement:
- Add a second chart (e.g., scatter of GDP vs life expectancy) that uses the same filters
- Add a checkbox to switch between ascending/descending sort
- Add a line chart for a selected country across years

Tip: In real dashboards, you’d keep filtering logic in one place to avoid mistakes.

---
## Summary and Key Takeaways

### What We Learned
In this chapter, we explored advanced visualization techniques that go beyond static charts:

| Topic | Key Points |
|-------|------------|
| **Static vs. Interactive** | Static charts are great for reports; interactive charts excel at exploration |
| **Interactive Concepts** | Hover, zoom, pan, selection, filtering, and linked views |
| **Plotly Express** | Quick interactive charts with minimal code |
| **Controls** | Dropdowns and sliders let users explore data without new code |
| **Dashboards** | Combine multiple charts with shared filters for decision support |
| **Geospatial** | Choropleth maps for regional data; use rates instead of totals |
| **Bokeh & Altair** | Alternative libraries with different strengths |
| **Exporting** | HTML for interactive sharing; PNG/SVG for static documents |

### Common Mistakes to Avoid
1. **Overusing interactivity** — Not every chart needs to be interactive
2. **Cluttered tooltips** — Show only the most relevant information on hover
3. **Fast animations** — They can hide details; use them for exploration, not final reporting
4. **Raw totals on maps** — Large regions dominate visually; prefer per-capita rates
5. **Too many dashboard elements** — Start with 2–3 key questions, not 20 charts

### Quick Reference: Library Comparison

| Library | Best For | Learning Curve |
|---------|----------|----------------|
| **Plotly** | General-purpose interactive charts, sharing as HTML | Easy |
| **Bokeh** | Dashboard apps, Python-first workflows | Medium |
| **Altair** | Exploratory analysis, clean declarative syntax | Easy-Medium |

### Additional Resources
- [Plotly Python Documentation](https://plotly.com/python/)
- [Plotly Dash (Dashboard Framework)](https://dash.plotly.com/)
- [Bokeh Documentation](https://docs.bokeh.org/)
- [Altair Documentation](https://altair-viz.github.io/)
- [Streamlit (Simple Dashboards)](https://streamlit.io/)

### What's Next?
In **Chapter 7: Exploratory Data Analysis and Descriptive Statistics**, we'll combine visualization with statistical methods to systematically explore and understand datasets. The interactive techniques you learned here will be valuable for EDA workflows.